<a href="https://colab.research.google.com/github/Leonard-Boante-Mensah/AIGES_VIDEO_SUMMARIZATION/blob/main/AIGES_VIDEO_SUMMARIZATION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import os
from PIL import Image
from pathlib import Path
import cv2
import h5py
import numpy as np
from tqdm import tqdm, trange
import argparse
import pdb
import glob
import math
import torch
from torch import nn
from torchvision import transforms, models
from torch.utils.data import DataLoader
import torch.optim as optim

import pprint
from collections import OrderedDict
from prettytable import PrettyTable
import json
from torch.utils.tensorboard import SummaryWriter

**A FUNCTION TO CONVERT VIDEO TO FRAMES**

In [ ]:
def Convert_Video2Frames(video_path, image_path):
  capture = cv2.VideoCapture(video_path)
  frame_seconds = 0.0831
  frameNumber = 1
    
  while (True):
      success, frame = capture.read()
      if success:
          cv2.imwrite(f'{image_path}img{frameNumber:05}.jpg', frame)
      else:
          break
      frame_seconds += 0.0831
      frameNumber += 1

**A FUNCTION TO PULL THE IMAGES INTO A SINGLE FOLDER**

In [ ]:
video_path = '/content/drive/MyDrive/Video_summarization_build/video_summarization_aiges/dataset/train/train_videos/'
img_path = '/content/drive/MyDrive/Video_summarization_build/video_summarization_aiges/dataset/train/output/'
h5_path = '/content/drive/MyDrive/Video_summarization_build/video_summarization_aiges/dataset/train/h5_path.h5'
# video_dir = Path(video_path).resolve()
# video_list = list(video_path.glob('*.mp4'))
# print(video_list.sort())
# print(video_path)
for data in os.listdir(video_path):
  dta = data.split('.')[-1]
  if dta == 'mp4':
    video_dir = f'{video_path}{data}'
    img_dir = f"{img_path}{data.split('.')[0]}/"
    Convert_Video2Frames(video_dir, img_dir)
#   video_dir = Path(data).resolve()
#    print(data)

**FUNCTION TO LOAD DATASET FROM GROUND TRUTH**

In [ ]:
def load_data(dataset):
  data = pd.read_excel(dataset, index_col=0)
  interest_col = data[['canSpeed']]
  interest_col = interest_col.apply(lambda x: abs(x))
  percentile_80th = interest_col['canSpeed'].quantile(0.80)
  interest_col['canSpeed'] = [1 if y > percentile_80th else 0 for y in interest_col['canSpeed']]
  data = interest_col['canSpeed']
  return data

**A FUNCTION TO GET THE LABELS OF THE VIDEOS**

In [ ]:
def get_speed_labels(dataset):
  data = load_data(dataset)
  labels = data.to_numpy()
  return labels

**CREATING OF PERSONAL SUMMARY VIDEOS FROM SPEED THRESHOLD**

In [ ]:
def loadVideoSummary(excel_path, frame_path, vid_path):
  # Loading the excel file into the pandaframe
  data = pd.read_excel(excel_path, index_col=0)
  interest_col = data[['canSpeed']]
  interest_col = interest_col.apply(lambda x: abs(x))
    
  # Change threshold values to 1 and 0
  percentile_80th = interest_col['canSpeed'].quantile(0.8)
  interest_col['canSpeed'] = [1 if y > percentile_80th else 0 for y in interest_col['canSpeed']]
  selected_columns = interest_col.loc[interest_col['canSpeed'] == 1]
  index_list = selected_columns.index.tolist()
  labels = [x[-12:] for x in index_list]
  labels.sort()
    
  # Load the selected images to the summarized video directory
  for img in labels:
      r_path = f'{frame_path}{img}'
      image = cv2.imread(r_path)
      cv2.imwrite(f'{vid_path}{img}', image)

WORKING WITH THE GROUND TRUTH TABLE

In [ ]:
# Reading of the raw excel file in a dataframe
dataset = pd.read_excel('/content/drive/MyDrive/Video_summarization_build/video_summarization_aiges/dataset/train/train_videos/Annotation_dataset.xlsx')
dataset.head()

,cameraRight,cameraFront,cameraRear,cameraLeft,here,tomtom,gpsLatitude,gpsLongitude,gpsAltitude,gpsPrecision,hereMmLatitude,hereMmLongitude,hereSpeedLimit,hereSpeedLimit_2,hereFreeFlowSpeed,hereSignal,hereYield,herePedestrian,hereIntersection,hereMmIntersection,hereSegmentExitHeading,hereSegmentEntryHeading,hereSegmentOthersHeading,hereCurvature,hereCurrentHeading,here1mHeading,here5mHeading,here10mHeading,here20mHeading,here50mHeading,hereTurnNumber,canSteering,canSpeed,chapter,abs_curv,abs_steer,norm_steer,norm_curv,norm_speed,score,label
0,Appenzell/go_pro_1/image/4/img00001.jpg,Appenzell/go_pro_4/image/4/img00001.jpg,Appenzell/go_pro_5/image/4/img00001.jpg,Appenzell/go_pro_8/image/4/img00001.jpg,Appenzell/here/image/4/img00001.jpg,Appenzell/tomtom/image/4/img00001.jpg,47.328652,8.795806,591.413975,138.0,47.328661,8.795782,50.0,NaN,37,NaN,NaN,NaN,13.979509,13.823462,28.753016,110.246236,314.526,0.004646,180.000000,151.246924,151.246926,151.246929,123.652396,157.135211,0,8.0,2.918944,138,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Appenzell/go_pro_1/image/4/img00002.jpg,Appenzell/go_pro_4/image/4/img00002.jpg,Appenzell/go_pro_5/image/4/img00002.jpg,Appenzell/go_pro_8/image/4/img00002.jpg,Appenzell/here/image/4/img00002.jpg,Appenzell/tomtom/image/4/img00002.jpg,47.328653,8.795807,591.407357,138.0,47.328662,8.795783,50.0,NaN,37,NaN,NaN,NaN,13.849176,13.689723,28.753016,110.246236,314.526,0.004780,28.753077,0.000000,0.000003,0.000005,-27.831429,5.933524,0,8.0,3.151089,138,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Appenzell/go_pro_1/image/4/img00003.jpg,Appenzell/go_pro_4/image/4/img00003.jpg,Appenzell/go_pro_5/image/4/img00003.jpg,Appenzell/go_pro_8/image/4/img00003.jpg,Appenzell/here/image/4/img00003.jpg,Appenzell/tomtom/image/4/img00003.jpg,47.328653,8.795807,591.417911,138.0,47.328662,8.795783,50.0,NaN,37,NaN,NaN,NaN,13.849176,13.689723,28.753016,110.246236,314.526,0.004780,28.753077,0.000000,0.000003,0.000005,-27.831429,5.933524,0,4.0,3.379237,138,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Appenzell/go_pro_1/image/4/img00004.jpg,Appenzell/go_pro_4/image/4/img00004.jpg,Appenzell/go_pro_5/image/4/img00004.jpg,Appenzell/go_pro_8/image/4/img00004.jpg,Appenzell/here/image/4/img00004.jpg,Appenzell/tomtom/image/4/img00004.jpg,47.328654,8.795808,591.430218,138.0,47.328663,8.795783,50.0,NaN,37,NaN,NaN,NaN,13.718921,13.555984,28.753016,110.246236,314.526,0.004914,28.753077,0.000001,0.000003,0.000005,-28.111442,5.978812,0,8.0,3.555985,138,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Appenzell/go_pro_1/image/4/img00005.jpg,Appenzell/go_pro_4/image/4/img00005.jpg,Appenzell/go_pro_5/image/4/img00005.jpg,Appenzell/go_pro_8/image/4/img00005.jpg,Appenzell/here/image/4/img00005.jpg,Appenzell/tomtom/image/4/img00005.jpg,47.328656,8.795810,591.429304,138.0,47.328665,8.795785,50.0,NaN,37,NaN,NaN,NaN,13.458649,13.288506,28.753016,110.246236,314.526,0.005181,28.753079,0.000001,0.000003,0.000006,-28.524311,6.097410,0,4.0,3.684775,138,NaN,NaN,NaN,NaN,NaN,NaN,NaN


***DATA PREPROCESSING STEPS***

In [ ]:
print(dataset.columns)

Index(['cameraRight', 'cameraFront', 'cameraRear', 'cameraLeft', 'here',
       'tomtom', 'gpsLatitude', 'gpsLongitude', 'gpsAltitude', 'gpsPrecision',
       'hereMmLatitude', 'hereMmLongitude', 'hereSpeedLimit',
       'hereSpeedLimit_2', 'hereFreeFlowSpeed', 'hereSignal', 'hereYield',
       'herePedestrian', 'hereIntersection', 'hereMmIntersection',
       'hereSegmentExitHeading', 'hereSegmentEntryHeading',
       'hereSegmentOthersHeading', 'hereCurvature', 'hereCurrentHeading',
       'here1mHeading', 'here5mHeading', 'here10mHeading', 'here20mHeading',
       'here50mHeading', 'hereTurnNumber', 'canSteering', 'canSpeed',
       'chapter', 'abs_curv', 'abs_steer', 'norm_steer', 'norm_curv',
       'norm_speed', 'score', 'label'],
      dtype='object')


In [ ]:
drop_columns = ['cameraRight', 'cameraFront','cameraRear', 'cameraLeft', 'here',
                'tomtom','gpsLatitude', 'gpsLongitude', 
                'gpsAltitude', 'gpsPrecision','hereMmLatitude', 
                'hereMmLongitude', 'hereSpeedLimit',
                'hereSpeedLimit_2', 'hereFreeFlowSpeed', 'hereSignal', 'hereYield',
                'herePedestrian', 'hereIntersection', 'hereMmIntersection',
                'hereSegmentExitHeading', 'hereSegmentEntryHeading',
                'hereSegmentOthersHeading', 'hereSegmentOthersHeading', 'hereCurvature', 'hereCurrentHeading',
                'here1mHeading', 'here5mHeading', 'here10mHeading', 'here20mHeading',
                'here50mHeading', 'hereTurnNumber', 'canSteering', 'chapter', 'abs_curv', 
                'abs_steer', 'norm_steer', 'norm_curv']

# Dropping of columns
data = dataset.drop(columns=drop_columns)

In [ ]:
data.head()

,canSpeed,norm_speed,score,label
0,2.918944,NaN,NaN,NaN
1,3.151089,NaN,NaN,NaN
2,3.379237,NaN,NaN,NaN
3,3.555985,NaN,NaN,NaN
4,3.684775,NaN,NaN,NaN


In [ ]:
data.describe()

,canSpeed,norm_speed,score,label
count,14990.000000,0.0,0.0,0.0
mean,12.593547,NaN,NaN,NaN
std,4.963667,NaN,NaN,NaN
min,0.000000,NaN,NaN,NaN
25%,10.501629,NaN,NaN,NaN
50%,13.198367,NaN,NaN,NaN
75%,16.346169,NaN,NaN,NaN
max,20.928110,NaN,NaN,NaN


In [ ]:
gt_score = data.canSpeed

**CREATING SUMMARY VIDEOS**

In [ ]:
path = '/content/drive/MyDrive/Video_summarization_build/video_summarization_aiges/dataset/train/output/'
w_path = '/content/drive/MyDrive/Video_summarization_build/video_summarization_aiges/dataset/train/summary_video/'
e_file = '/content/drive/MyDrive/Video_summarization_build/video_summarization_aiges/dataset/train/train_videos/'
video_path = '/content/drive/MyDrive/Video_summarization_build/video_summarization_aiges/dataset/train/train_videos/'

In [ ]:
for data in os.listdir(video_path):
  dta = data.split('.')[-1]
  if dta == 'xlsx':
      excel_file = f"{e_file}{data}"
      frame_path = f"{path}{data.split('.')[0]}/"
      video_dir = f"{w_path}{data.split('.')[0]}/"
      loadVideoSummary(excel_file, frame_path, video_dir)

#        video_dir = f'{video_path}{data}'
#        img_dir = f"{img_path}{data.split('.')[0]}/"
#        Convert_Video2Frames(video_dir, img_dir)

**CONVERTING VIDEOS TO KEY FEATURES FOR THE MODEL TRAINING**

RESCALING OF THE SIZE OF THE IMAGE

In [ ]:
class Rescale(object):
    """Rescale a image to a given size.

    Args:
        output_size (tuple or int): Desired output size. If tuple, output is matched to output_size. If int, smaller of image edges is matched to output_size keeping aspect ratio the same.
    """
    def __init__(self, *output_size):
        self.output_size = output_size

    def __call__(self, image):
        """
        Args:
            image (PIL.Image) : PIL.Image object to rescale
        """
        new_h, new_w = self.output_size
        new_h, new_w = int(new_h), int(new_w)
        img = image.resize((new_w, new_h), resample=Image.BILINEAR)
        return img

GETTING THE FEATURES OF THE GOOGLENET PRETRAINED MODEL

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# transformation
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),  # HWC->CHW [0,255]->[0.0,1.0]
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# googlenet preparation
net = models.googlenet(pretrained=True)


# we only want features with no grads
for param in net.parameters():
    param.requires_grad = False
net.to(device) # to GPU or CPU
fea_net = nn.Sequential(*list(net.children())[:-2]) # pool5 feature
fea_net.eval() # eval mode

In [ ]:
cps = []
x, y = 0, 100
while y < 14991:
    cps.append((x, y))
    x = y + 1
    y = y + 100

x, y = 14901, 14990
cps.append((x, y))

weight = np.ones(149)
weight = weight * 100
new_weight = np.append(weight, 90)

In [ ]:
import numpy as np

def knapsack(v, w, max_weight):
    rows = len(v) + 1
    cols = max_weight + 1

    # adding dummy values as later on we consider these values as indexed from 1 for convinence
    
    v = np.r_[[0], v]
    w = np.r_[[0], w]

    # row : values , #col : weights
    dp_array = [[0 for i in range(cols)] for j in range(rows)]

    # 0th row and 0th column have value 0

    # values
    for i in range(1, rows):
        # weights
        for j in range(1, cols):
            # if this weight exceeds max_weight at that point
            if j - w[i] < 0:
                dp_array[i][j] = dp_array[i - 1][j]

            # max of -> last ele taken | this ele taken + max of previous values possible
            else:
                dp_array[i][j] = max(dp_array[i - 1][j], v[i] + dp_array[i - 1][j - w[i]])

    # return dp_array[rows][cols]  : will have the max value possible for given wieghts

    chosen = []
    i = rows - 1
    j = cols - 1

    # Get the items to be picked
    while i > 0 and j > 0:

        # ith element is added
        if dp_array[i][j] != dp_array[i - 1][j]:
            # add the value
            chosen.append(i-1)
            # decrease the weight possible (j)
            j = j - w[i]
            # go to previous row
            i = i - 1

        else:
            i = i - 1

    return dp_array[rows - 1][cols - 1], chosen

**CREATING THE FEATURES AND LABELS TO FEED THE MODEL**

In [ ]:
def videoTwofeatures(video_path, cps, gt_score):

    # for feature and label
    video = cv2.VideoCapture(video_path)
    index = video_path.split('.')[0].split('/')[-1]
    tqdm.write('Processing video '+index)
    length = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    ratio = round(length / 1499)  

    # Handle the Labels
    cps = cps
    weight = list(new_weight)
    weight = [int(w) for w in weight]
    gtscore = np.ravel(gt_score.transpose())
    n_frame = 14990
    value = np.array([gtscore[cp[0]:(cp[1]+1)].mean() for cp in cps])
    _, selected = knapsack(value, weight, int(0.30*length))
    selected = selected[::-1]

    key_shots = np.zeros(shape=(n_frame, ))
    key_frames = np.zeros(shape=(n_frame, ))

    for i in selected:
        key_shots[cps[i][0]:(cps[i][1]+1)] = 1
        max_idx = np.where(gtscore[cps[i][0]:(cps[i][1]+1)] == np.max(gtscore[cps[i][0]:(cps[i][1]+1)]))
        max_idx = max_idx[0]
        key_frames_slice = key_frames[cps[i][0]:(cps[i][1]+1)]
        key_frames_slice[max_idx] = 1
        
    # EXTRACT FEATURES
    feature = []
    label = []
    i = 0
    success, frame = video.read()
    while success:
        if (i+1) % ratio == 0:
            frame_RGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame_RGB_PIL = Image.fromarray(frame_RGB)
            frame_RGB_PIL_transform = transform(frame_RGB_PIL).to(device).view(1,3,224,224)
            fea_out = fea_net(frame_RGB_PIL_transform).permute(0,3,2,1).contiguous().view(1024).detach().to("cpu")
            feature.append(fea_out)
            label.append(key_frames[i])
        i += 1
        success, frame = video.read()
        
    # FORMING THE DATASET INTO A STACK USING PYTORCH
    feature = torch.stack(feature, 0)
    feature = feature[:1499]
    label = label[:1499]
    return feature, label, cps, weight, n_frame

In [ ]:
video_path = '/content/drive/MyDrive/Video_summarization_build/video_summarization_aiges/dataset/train/train_videos/joined_appenzel.mp4'

In [ ]:
feature, label, cps, n_frame_per_seg, length = videoTwofeatures(video_path, cps, gt_score)

Processing video joined_appenzel


**MAKING OF THE DATASET**

In [ ]:
h5_data = '/content/drive/MyDrive/Video_summarization_build/video_summarization_aiges/dataset/train/train_videos/h5_video.h5'

In [ ]:
with h5py.File(h5_data, 'w') as h5_f:
    video_data = h5_f.create_group('training_video')
    video_data['feature'] = feature.numpy()
    video_data['label'] = label
    video_data['length'] = length
    video_data['change_points'] = cps
    video_data['n_frame_per_seg'] = n_frame_per_seg
h5_f.close()

**BUILDING THE DATASET**

In [ ]:
def build_dataset(video_dir, summary_dir, hd5_path):
  video_dir = Path(video_dir).resolve()
  video_list = list(video_dir.glob('*.mp4'))
  video_list.sort()
    
  # Creating the HDF5 folder for the dataset
  with h5py.File(hd5_path, 'w') as h5_f:
      for video_path in tqdm(video_list, desc='Video', ncols=80, leave=False):
          videoTwofeatures(video_path, summary_dir, h5_f)

In [ ]:
build_dataset(video_path, selected_frames, hd5_path)

**CREATING THE TRAINING AND TESTING DATASET**

In [ ]:
# CREATING A CLASS FOR THE VIDEO DATASET
class VideoData(object):
    def __init__(self, data_path):
        self.data_file = h5py.File(data_path)

    def __len__(self):
        return len(self.data_file)

    def __getitem__(self, train_data):
        video = self.data_file['training_video']
        feature = video_feature = torch.from_numpy(video["feature"][...]).transpose(1,0).view(1024,-1)
        label = torch.from_numpy(video["label"][...]).type(torch.long)
        return feature, label

**LOADER FUNCTION**

In [ ]:
def get_loader(path, batch_size=5):
    # Loading the training dataset
    dataset = VideoData(path)
    train_dataset, test_dataset = torch.utils.data.random_split(dataset, [len(dataset) - len(dataset) // 5, len(dataset) // 5])
    train_loader = DataLoader(train_dataset, batch_size=batch_size)
    return train_loader, test_dataset

    # train_loader_list = []
    # test_dataset_list = []

    # split_times = 5

    # for i in range(split_times):
    #     train_dataset,test_dataset = torch.utils.data.random_split(dataset, [int(dataset.__len__()*0.8), int(dataset.__len__()*0.2)])

    #     # shuffle (bool, optional): set to `True` to have the data reshuffled at every epoch
    #     train_loader = torch.utils.data.DataLoader(train_dataset, batch_size, shuffle=True)

    #     train_loader_list.append(train_loader)
    #     test_dataset_list.append(test_dataset)

    # return train_loader_list, test_dataset_list, dataset.data_file

**CREATE A CONFIGURATION TO SAVE MODEL SCORES AND PREDICTIONS**

In [ ]:
class Config():
  """Config class"""
  def __init__(self, **kwargs):
      # Path
      self.data_path = '/content/drive/MyDrive/Video_summarization_build/video_summarization_aiges/dataset/train/h5_path.h5'
      self.save_dir = '/content/drive/MyDrive/Video_summarization_build/video_summarization_aiges/dataset/save_dir'
      self.score_dir = '/content/drive/MyDrive/Video_summarization_build/video_summarization_aiges/dataset/score_dir'
      self.log_dir = '/content/drive/MyDrive/Video_summarization_build/video_summarization_aiges/dataset/log_dir'

      # Model
      self.mode = 'train'
      self.gpu = True
      self.n_epochs = 5
      self.n_class = 2
      self.lr = 1e-3
      self.momentum = 0.9
      self.batch_size = 5

      for k, v in kwargs.items():
        setattr(self, k, v)

  def __repr__(self):
    config_str = 'Configurations\n' + pprint.pformat(self.__dict__)
    return config_str

In [ ]:
config = Config()
print(config)

Configurations
{'batch_size': 5,
 'data_path': '/content/drive/MyDrive/Video_summarization_build/video_summarization_aiges/dataset/train/h5_path.h5',
 'gpu': True,
 'log_dir': '/content/drive/MyDrive/Video_summarization_build/video_summarization_aiges/dataset/log_dir',
 'lr': 0.001,
 'mode': 'train',
 'momentum': 0.9,
 'n_class': 2,
 'n_epochs': 5,
 'save_dir': '/content/drive/MyDrive/Video_summarization_build/video_summarization_aiges/dataset/save_dir',
 'score_dir': '/content/drive/MyDrive/Video_summarization_build/video_summarization_aiges/dataset/score_dir'}


**CREATING THE FULLY CONVOLUTIONAL SEMANTIC NETWORK**

In [ ]:
class FCSN(nn.Module):
  def __init__(self, n_class=2):
    super(FCSN, self).__init__()

    self.conv1 = nn.Sequential(OrderedDict([
      ('conv1_1', nn.Conv1d(1024, 1024, 3, padding=1)),
      ('bn1_1', nn.BatchNorm1d(1024)),
      ('relu1_1', nn.ReLU(inplace=True)),
      ('conv1_2', nn.Conv1d(1024, 1024, 3, padding=1)),
      ('bn1_2', nn.BatchNorm1d(1024)),
      ('relu1_2', nn.ReLU(inplace=True)),
      ('pool1', nn.MaxPool1d(2, stride=2, ceil_mode=True))
      ])) # 1/2

    self.conv2 = nn.Sequential(OrderedDict([
      ('conv2_1', nn.Conv1d(1024, 1024, 3, padding=1)),
      ('bn2_1', nn.BatchNorm1d(1024)),
      ('relu2_1', nn.ReLU(inplace=True)),
      ('conv2_2', nn.Conv1d(1024, 1024, 3, padding=1)),
      ('bn2_2', nn.BatchNorm1d(1024)),
      ('relu2_2', nn.ReLU(inplace=True)),
      ('pool2', nn.MaxPool1d(2, stride=2, ceil_mode=True))
      ])) # 1/4

    self.conv3 = nn.Sequential(OrderedDict([
      ('conv3_1', nn.Conv1d(1024, 1024, 3, padding=1)),
      ('bn3_1', nn.BatchNorm1d(1024)),
      ('relu3_1', nn.ReLU(inplace=True)),
      ('conv3_2', nn.Conv1d(1024, 1024, 3, padding=1)),
      ('bn3_2', nn.BatchNorm1d(1024)),
      ('relu3_2', nn.ReLU(inplace=True)),
      ('conv3_3', nn.Conv1d(1024, 1024, 3, padding=1)),
          ('bn3_3', nn.BatchNorm1d(1024)),
          ('relu3_3', nn.ReLU(inplace=True)),
          ('pool3', nn.MaxPool1d(2, stride=2, ceil_mode=True))
          ])) # 1/8

    self.conv4 = nn.Sequential(OrderedDict([
        ('conv4_1', nn.Conv1d(1024, 2048, 3, padding=1)),
        ('bn4_1', nn.BatchNorm1d(2048)),
        ('relu4_1', nn.ReLU(inplace=True)),
        ('conv4_2', nn.Conv1d(2048, 2048, 3, padding=1)),
        ('bn4_2', nn.BatchNorm1d(2048)),
        ('relu4_2', nn.ReLU(inplace=True)),
        ('conv4_3', nn.Conv1d(2048, 2048, 3, padding=1)),
        ('bn4_3', nn.BatchNorm1d(2048)),
        ('relu4_3', nn.ReLU(inplace=True)),
        ('pool4', nn.MaxPool1d(2, stride=2, ceil_mode=True))
        ])) # 1/16

    self.conv5 = nn.Sequential(OrderedDict([
        ('conv5_1', nn.Conv1d(2048, 2048, 3, padding=1)),
        ('bn5_1', nn.BatchNorm1d(2048)),
        ('relu5_1', nn.ReLU(inplace=True)),
        ('conv5_2', nn.Conv1d(2048, 2048, 3, padding=1)),
        ('bn5_2', nn.BatchNorm1d(2048)),
        ('relu5_2', nn.ReLU(inplace=True)),
        ('conv5_3', nn.Conv1d(2048, 2048, 3, padding=1)),
        ('bn5_3', nn.BatchNorm1d(2048)),
        ('relu5_3', nn.ReLU(inplace=True)),
        ('pool5', nn.MaxPool1d(2, stride=2, ceil_mode=True))
        ])) # 1/32

    self.conv6 = nn.Sequential(OrderedDict([
        ('fc6', nn.Conv1d(2048, 4096, 1)),
        ('bn6', nn.BatchNorm1d(4096)),
        ('relu6', nn.ReLU(inplace=True)),
        ('drop6', nn.Dropout())
        ]))
   
    self.conv7 = nn.Sequential(OrderedDict([
        ('fc7', nn.Conv1d(4096, 4096, 1)),
        ('bn7', nn.BatchNorm1d(4096)),
        ('relu7', nn.ReLU(inplace=True)),
        ('drop7', nn.Dropout())
        ]))

    self.conv8 = nn.Sequential(OrderedDict([
        ('fc8', nn.Conv1d(4096, n_class, 1)),
        ('bn8', nn.BatchNorm1d(n_class)),
        ('relu8', nn.ReLU(inplace=True)),
        ]))

    self.conv_pool4 = nn.Conv1d(2048, n_class, 1)
    self.bn_pool4 = nn.BatchNorm1d(n_class)

    self.deconv1 = nn.ConvTranspose1d(n_class, n_class, 4, padding=1, stride=2, bias=False)
    self.deconv2 = nn.ConvTranspose1d(n_class, n_class, 16, stride=16, bias=False)

  def forward(self, x):
    h = x
    h = self.conv1(h)
    h = self.conv2(h)
    h = self.conv3(h)
    h = self.conv4(h)
    pool4 = h

    h = self.conv5(h)
    h = self.conv6(h)
    h = self.conv7(h)
    h = self.conv8(h)

    h = self.deconv1(h)
    upscore2 = h

    h = self.conv_pool4(pool4)
    h = self.bn_pool4(h)
    score_pool4 = h

    h = upscore2 + score_pool4

    h = self.deconv2(h)

    # h_softmax = self.softmax(h) # [5,2,320]
    # mask = h_softmax[:,1,:].view(-1,1,1499) # [5,1,320] use key frame score to be the mask

    # h_reconstruct = self.relu_reconstuct1(self.bn_reconstruct1(self.conv_reconstuct1(h))) # [5,1024,320]

    # # merge with input features
    # h_merge = h_reconstruct + x # [5,1024,1499]
    # h_merge_reconstruct = self.relu_reconstuct2(self.bn_reconstruct2(self.conv_reconstuct2(h_merge))) # [5,1024,320]

    # return h_merge_reconstruct, mask, h # [5,1024,320], [5,1,320], [5,2,320]

    return h

**UPSAMPLING OF THE FRAMES IN THE VIDEO**

In [ ]:
def upsample(down_arr, N):
  up_arr = np.zeros(N)
  ratio = N // 1499
  l = (N - ratio * 1499) // 2
  i = 0
  while i < 1499:
    up_arr[l:l+ratio] = np.ones(ratio, dtype=int) * down_arr[i]
    l += ratio
    i += 1
  return up_arr

In [ ]:
net = FCSN()
data = torch.randn((1, 1024, 1499))
print(net(data).shape)

torch.Size([1, 2, 1504])


**CACULATING PREDICTION SCORE**

In [ ]:
def select_keyshots(video_info, pred_score):
    N = video_info['length'][()]
    cps = video_info['change_points'][()]
    weight = video_info['n_frame_per_seg'][()]
    pred_score = np.array(pred_score.cpu().data)
    pred_score = upsample(pred_score, N)

    pred_value = np.array([pred_score[cp[0]:cp[1]].mean() for cp in cps])
    _, selected = knapsack(pred_value, weight, int(0.15 * N))
    selected = selected[::-1]
    key_labels = np.zeros(shape=(N, ))
    for i in selected:
        key_labels[cps[i][0]:cps[i][1]] = 1
    return pred_score.tolist(), selected, key_labels.tolist()
  

## **CREATE CLASS FOR TRAINING THE MODEL**

In [ ]:
class Solver(object):
    def __init__(self, config=None, train_loader=None, test_dataset=None):

        """Class that Builds, Trains FCSN model"""
        self.config = config
        self.train_loader = train_loader
        self.test_dataset = test_dataset

        # model
        self.model = FCSN(self.config.n_class)

        # optimizer
        if self.config.mode == 'train':
            self.optimizer = optim.Adam(self.model.parameters())
            # self.optimizer = optim.SGD(self.model.parameters(), lr=config.lr, momentum=self.config.momentum)
            self.model.train()

        if self.config.gpu:
            self.model = self.model.cuda()

        if not os.path.exists(self.config.score_dir):
            os.mkdir(self.config.score_dir)

        if not os.path.exists(self.config.save_dir):
            os.mkdir(self.config.save_dir)

        if not os.path.exists(self.config.log_dir):
            os.mkdir(self.config.log_dir)
    
    @staticmethod
    def sum_loss(pred_score, gt_labels, weight=None):
        n_batch, n_class, n_frame = pred_score.shape
        log_p = torch.log_softmax(pred_score, dim=1).reshape(-1, n_class)
        gt_labels = gt_labels.reshape(-1)
        criterion = torch.nn.NLLLoss(weight)
        loss = criterion(log_p, gt_labels)
        return loss

    def train(self):
        writer = SummaryWriter(log_dir=self.config.log_dir)
        t = trange(self.config.n_epochs, desc='Epoch', ncols=80)

        for epoch_i in t:
            sum_loss_history = []
            for batch_i, (feature, label) in enumerate(tqdm(self.train_loader, desc='Batch', ncols=80, leave=False)):
                # Hey yah
                # [batch_size, 1024, seq_len]
                feature.requires_grad_()
                # => cuda
                if self.config.gpu:
                    feature = feature.cuda()
                    label = label

                # ------- Train --------- #
                pred_score = self.model(feature)
                label_1 = label.sum() / label.shape[0]
                label_0 = label.shape[1] - label_1
                weight = torch.tensor([label_1, label_0], dtype=torch.float)

                if self.config.gpu:
                    weight = weight.cuda()

                loss = self.sum_loss(pred_score, label, weight)
                loss.backward()

                self.optimer.step()
                self.optimizer.zero_grad()
                sum_loss_history.append(loss)
                print(sum_loss_history)

            mean_loss = torch.stack(sum_loss_history).mean().item()
            t.set_postfix(loss=mean_loss)
            writer.add_scalar('Loss', mean_loss, epoch_i)

            if (epoch_i+1) % 5 == 0:
                # Calculating the Epoch
                ckpt_path = self.config.save_dir + '/epoch-{}.pkl'.format(epoch_i)
                tqdm.write('Save parameters at {}'.format(ckpt_path))
                torch.save(self.model.state_dict(), ckpt_path)
                self.evaluate(epoch_i)
                self.model.train()

    def evaluate(self, epoch_i):
        self.model.eval()
        out_dict = {}
        eval_arr = []
        table = PrettyTable()
        table.title = f'Evaluation Result of Epoch {epoch_i}'
        table.field.names = ['ID', 'Precision', 'Recall', 'F-score']
        table.float_format = '1.3'

        with h5py.File(self.config.data_path) as data_file:
            for feature, label, idx in tqdm(self.test_dataset, desc='Evaluate', ncols=80, leave=False):
                if self.config.gpu:
                    feature = feature.cuda()
                    pred_score = self.model(feature.unqueeze(0).squeeze(0))
                    pred_score = torch.softmax(pred_score, dim=0)[1]
                    video_info = data_file['training_video']
                
                pred_score, pred_selected, pred_summary = select_keyshots(video_info, pred_score)

                out_dict[idx] = {
                    'pred_score': pred_score,
                    'pred_selected': pred_selected,
                    'pred_summary': pred_summary
                }

        score_save_path = self.config.score_dir + '/epoch-{}.json'.format(epoch_i)
        with open(score_save_path, 'w') as f:
            tqdm.write('Save score at {}'.format(str(score_save_path)))
            json.dump(out_dict, f)
        tqdm.write(str(table))

In [ ]:
def sum_loss(pred_score, gt_labels, weight=None):
    n_batch, n_class, n_frame = pred_score.shape
    log_p = torch.log_softmax(pred_score, dim=1).reshape(-1, n_class)
    gt_labels = gt_labels.reshape(-1)
    criterion = torch.nn.NLLLoss(weight)
    loss = criterion(log_p, gt_labels)
    return loss

In [ ]:
optimizer = optim.Adam(model.parameters())

In [ ]:
for epoch_i in t:
    sum_loss_history = []
    for batch_i, (feature, label) in enumerate(tqdm(train_loader_list, desc='Batch', ncols=80, leave=False)):
        feature.requires_grad_()

        if config.gpu:
            feature = feature.cuda()
            label = label.to(device)
        pred_score = model(feature)
        print(pred_score)
        label_1 = label.sum() / label.shape[0]
        label_0 = label.shape[1] - label_1
        weight = torch.tensor([label_1, label_0], dtype=torch.float)

        if config.gpu:
            weight = weight.cuda()
        print(pred_score.shape)
        print(label.shape)

        # loss = sum_loss(pred_score, label, weight)
        # loss.backward()

        # print(loss)

        # optimizer.step()
        # optimizer.zero_grad()
        # sum_loss_history.append(loss)

        # n_batch, n_class, f_frame = pred_score.shape
        # print(n_class)

        

tensor([[[-0.0726,  0.0633, -0.1778,  ..., -0.1156,  0.1799, -0.1160],
         [ 0.0897, -0.0598,  0.0993,  ..., -0.1700, -0.0044,  0.1303]]],
       device='cuda:0', grad_fn=<SqueezeBackward1>)
torch.Size([1, 2, 1504])
torch.Size([1, 1499])


tensor([[[-0.0677,  0.0611, -0.1711,  ..., -0.1165,  0.1993, -0.1111],
         [ 0.0860, -0.0563,  0.0942,  ..., -0.1746,  0.0195,  0.1305]]],
       device='cuda:0', grad_fn=<SqueezeBackward1>)
torch.Size([1, 2, 1504])
torch.Size([1, 1499])


tensor([[[-0.0774,  0.0623, -0.1765,  ..., -0.1156,  0.1799, -0.1160],
         [ 0.0898, -0.0624,  0.1018,  ..., -0.1700, -0.0044,  0.1303]]],
       device='cuda:0', grad_fn=<SqueezeBackward1>)
torch.Size([1, 2, 1504])
torch.Size([1, 1499])


tensor([[[-0.0915,  0.0640, -0.1843,  ..., -0.1156,  0.1799, -0.1160],
         [ 0.0953, -0.0711,  0.1128,  ..., -0.1700, -0.0044,  0.1303]]],
       device='cuda:0', grad_fn=<SqueezeBackward1>)
torch.Size([1, 2, 1504])
torch.Size([1, 1499])


tensor([[[-0.0677,  0.0611, -0.1711,  ..., -0.1073,  0.1843, -0.1021],
         [ 0.0860, -0.0563,  0.0942,  ..., -0.1609,  0.0189,  0.1202]]],
       device='cuda:0', grad_fn=<SqueezeBackward1>)
torch.Size([1, 2, 1504])
torch.Size([1, 1499])


**TESTING OF THE MODEL**

In [ ]:
train_config = Config()
test_config = Config(mode='test')
train_loader, test_dataset = get_loader(train_config.data_path, batch_size=train_config.batch_size)
solver = Solver(train_config, train_loader, test_dataset)

In [ ]:
print(train_config)

Configurations
{'batch_size': 5,
 'data_path': '/content/drive/MyDrive/Video_summarization_build/video_summarization_aiges/dataset/train/h5_path.h5',
 'gpu': True,
 'log_dir': '/content/drive/MyDrive/Video_summarization_build/video_summarization_aiges/dataset/log_dir',
 'lr': 0.001,
 'mode': 'train',
 'momentum': 0.9,
 'n_class': 2,
 'n_epochs': 5,
 'save_dir': '/content/drive/MyDrive/Video_summarization_build/video_summarization_aiges/dataset/save_dir',
 'score_dir': '/content/drive/MyDrive/Video_summarization_build/video_summarization_aiges/dataset/score_dir'}


In [ ]:
test_config = Config(mode='test')

In [ ]:
print(test_config)

Configurations
{'batch_size': 5,
 'data_path': '/content/drive/MyDrive/Video_summarization_build/video_summarization_aiges/dataset/train/h5_path.h5',
 'gpu': True,
 'log_dir': '/content/drive/MyDrive/Video_summarization_build/video_summarization_aiges/dataset/log_dir',
 'lr': 0.001,
 'mode': 'test',
 'momentum': 0.9,
 'n_class': 2,
 'n_epochs': 5,
 'save_dir': '/content/drive/MyDrive/Video_summarization_build/video_summarization_aiges/dataset/save_dir',
 'score_dir': '/content/drive/MyDrive/Video_summarization_build/video_summarization_aiges/dataset/score_dir'}


In [ ]:
train_loader, test_dataset = get_loader(train_config.data_path, batch_size=train_config.batch_size)

In [ ]:
print(test_dataset)

In [ ]:
solver = Solver(train_config, train_loader, test_dataset)

**FUNCTION TO CONVERT VIDEO DATA TO FRAMES**

In [ ]:
def Convert_Video2Frames(video_path, image_path):
  capture = cv2.VideoCapture(video_path)
  frame_seconds = 0.0831
  frameNumber = 1

  while (True):
    success, frame = capture.read()
    if success:
      cv2.imwrite(f'{image_path}img{frameNumber:05}.jpg', frame)
    else:
      break
    frame_seconds += 0.0831
    frameNumber += 1

In [ ]:
video_path = 'C:/Users/joepr/Documents/AIGES_CAPSTONE_PROJECT/video_summarization_aiges/dataset/train/train_videos/'
img_path = 'C:/Users/joepr/Documents/AIGES_CAPSTONE_PROJECT/video_summarization_aiges/dataset/train/output/'
h5_path = 'C:/Users/joepr/Documents/AIGES_CAPSTONE_PROJECT/video_summarization_aiges/dataset/train/h5_path'
# video_dir = Path(video_path).resolve()
# video_list = list(video_path.glob('*.mp4'))
# print(video_list.sort())
# print(video_path)
for data in os.listdir(video_path):
    dta = data.split('.')[-1]
    if dta == 'mp4':
        video_dir = f'{video_path}{data}'
        img_dir = f"{img_path}{data.split('.')[0]}/"
        Convert_Video2Frames(video_dir, img_dir)
#     video_dir = Path(data).resolve()
#     print(data)

**FUNCTION TO PREPARE EXCEL DATASET**

In [ ]:
def prepare_dataset(data_path):
  data = pd.read_excel(data_path, index_col=0)
  col = data[['canSpeed']]
  col = col.apply(lambda x: abs(x))
  max_speed = col['canSpeed'].max()
  mean_speed = col['canSpeed'].mean()
  col['mean_speed'] = [1 if y > mean_speed else 0 for y in col['canSpeed']]
  col['max_speed'] = [1 if y > max_speed else 0 for y in col['canSpeed']]
  mean_selected = col[col['mean_speed'] == 1]
  max_selected = col[col['max_speed'] == 1]
  mean_selected = mean_selected.index.tolist()
  max_selected = max_selected.index.tolist()
  mean_labels = [row[-12:] for row in mean_selected]
  max_labels = [row[-12:] for row in max_selected]
  mean_labels.sort()
  max_labels.sort()
  return mean_labels, max_labels

**WRITE SELECTED FRAMES INTO ANOTHER FOLDER**

In [ ]:
def selected_frames(path1, path2, sel_images):
  for img in sel_images:
    path = f'{path1}{img}'
    image = cv2.imread(path)
    cv2.imwrite('f{path2}{img}', image)